In [ ]:
import os
import urllib.request
import tarfile
import shutil
import glob

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# 17flowersデータセットのダウンロード
urllib.request.urlretrieve(
    'http://www.robots.ox.ac.uk/~vgg/data/flowers/17/17flowers.tgz',
    '17flowers.tgz'
)

In [ ]:
# 17flowersデータセットの解凍
with tarfile.open('17flowers.tgz') as tar:
    tar.extractall()
os.rename('jpg', '17flowers')

In [ ]:
# 17flowersデータセットのラベルを定義
labels = ['Tulip', 'Snowdrop', 'LilyValley', 'Bluebell', 'Crocus', 
          'Iris', 'Tigerlily', 'Daffodil', 'Fritillary', 'Sunflower', 
          'Daisy', 'ColtsFoot', 'Dandelion', 'Cowslip', 'Buttercup', 
          'Windflower', 'Pansy']

In [ ]:
# train/validationディレクトリパス準備
train_dir = os.path.join(os.getcwd(), '17flowers', 'train')
validation_dir = os.path.join(os.getcwd(), '17flowers', 'validation')

In [ ]:
# train/validationに各ラベルのディレクトリを準備
os.mkdir(train_dir)
os.mkdir(validation_dir)

for directory_name in labels:
    os.mkdir(os.path.join(os.getcwd(), '17flowers', 'train', directory_name))
    os.mkdir(os.path.join(os.getcwd(), '17flowers', 'validation', directory_name))

In [ ]:
# train/validationにデータセットを配置
dataset_number = 80
train_ratio = 0.75
train_number = int(dataset_number * train_ratio)

jpg_files = [f for f in sorted(os.listdir('17flowers')) if f.endswith('.jpg')]
for index, jpg_file in enumerate(jpg_files):
    if (index % dataset_number) < train_number:
        destination_directory = 'train'
    else:
        destination_directory = 'validation'
        
    src = os.path.join(os.getcwd(), '17flowers', jpg_file)
    dst = os.path.join(os.getcwd(), '17flowers', destination_directory, labels[index // dataset_number])
    shutil.move(src, dst)

In [ ]:
# train/validation配下のjpgファイル一覧を取得
train_files = glob.glob(os.path.join(train_dir, '*', '*.jpg'))
validation_files = glob.glob(os.path.join(validation_dir, '*', '*.jpg'))

In [ ]:
# VGG16モデルをロード(include_top=False:ネットワークの出力層側にある全結合層を含まない)
vgg16_base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

In [ ]:
# モデル可視化
vgg16_base_model.summary()

In [ ]:
# 15層までを固定し再学習しないよう設定
for layer in vgg16_base_model.layers[:15]:
    layer.trainable = False
for layer in vgg16_base_model.layers:
    print(layer, layer.trainable)

In [ ]:
# 出力用の全結合層を再定義しモデルを構築
x = tf.keras.layers.Flatten()(vgg16_base_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
output = tf.keras.layers.Dense(17, activation='softmax', name='last_output')(x)
vgg16_model = tf.keras.Model(inputs=vgg16_base_model.inputs, outputs=output, name='model')

In [ ]:
# モデルコンパイル(転移学習、ファインチューニングの場合はAdamよりSGDが良いケースが多い)
vgg16_model.compile(
    optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# モデル可視化
vgg16_model.summary()

In [ ]:
# train/validation用のImageDataGeneratorを定義(データ拡張をする場合はコメントアウトを解除する)
train_image_generator = ImageDataGenerator(rescale=1./255,
                                           # width_shift_range=2,
                                           # height_shift_range=2,
                                           # brightness_range=(0.8, 1.2),
                                           # channel_shift_range=0.2,
                                           # zoom_range=0.02,
                                           # rotation_range=2
                                          )
validation_image_generator = ImageDataGenerator(rescale=1.0/255)

In [ ]:
# ImageDataGeneratorを用いてディレクトリからデータを読み込む準備
batch_size = 128
epochs = 30
IMG_HEIGHT = 224
IMG_WIDTH = 224

train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           classes=labels,
                                                           class_mode='categorical')
validation_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              classes=labels,
                                                              class_mode='categorical')

In [ ]:
# モデルチェックポイントのコールバック(1エポック毎)
checkpoint_path = os.path.join(os.getcwd(), 'checkpoints', 'weights.{epoch:03d}-{val_loss:.3f}-{val_accuracy:.3f}.hdf5')
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 verbose=1, 
                                                 save_best_only=True,
                                                 mode='auto',
                                                 save_weights_only=False, 
                                                 save_freq='epoch')

# 評価値の改善が見られない場合に学習率を減らすコールバックを定義
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5,
                                          verbose=1, mode='auto', min_delta=0.0001,
                                          cooldown=3, min_lr=0)

In [ ]:
# 訓練
with tf.device("CPU:0"): # CUDA、cuDNNが正しくインストールされている場合はwith句を外す
    history = vgg16_model.fit(
        train_data_gen,
        steps_per_epoch=len(train_files) // batch_size,
        epochs=epochs,
        validation_data=validation_data_gen,
        validation_steps=len(validation_files) // batch_size,
        callbacks=[cp_callback, lr_callback]
        # callbacks=[cp_callback]
    )

In [ ]:
# 保存したモデルのロード
load_model = tf.keras.models.load_model("checkpoints/weights.***-****-****.hdf5") # 出来上がったcheckpointファイルを指定する

In [ ]:
# テスト画像を1枚ロード
from IPython.display import Image, display_png
from tensorflow.keras.preprocessing.image import img_to_array, load_img

img = tf.keras.preprocessing.image.load_img('17flowers/validation/Sunflower/image_0781.jpg', False, target_size=(224, 224))
display_png(img)

In [ ]:
# 入力画像成形、および正規化
x = img_to_array(img)
x = x.reshape(-1, 224, 224, 3)
x = x.astype('float32')
x /= 255

In [ ]:
# 推論実行
with tf.device("CPU:0"): # CUDA、cuDNNが正しくインストールされている場合はwith句を外す
    predict_result = load_model.predict(x)

In [ ]:
# 推論結果表示
print(predict_result)
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))
print(labels[np.argmax(np.squeeze(predict_result))])